In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.anova import AnovaRM
from statsmodels.graphics.factorplots import interaction_plot

from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pingouin as pg
from pingouin import ttest

from CosinorPy import file_parser, cosinor, cosinor1

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn import metrics

#dimensionality reduction
from sklearn.decomposition import PCA

#clustering
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc
from bioinfokit.visuz import cluster

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sns.set_context("poster")
sns.set_style("ticks")
sns.set(font_scale=2)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

# Get data

## Get filtered drinking data combined with RFID (already processed in other nb)

In [ ]:
path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/polysubstance_paper/data/final/clean/VDM/VDM_poly_final_clean_231103.csv'
data_VDM_start = pd.read_csv(path)
data_VDM_start = pd.DataFrame(data = data_VDM_start)
data_VDM_start['Id'] = data_VDM_start['Sex'] + '_' + data_VDM_start['IdRFID'].astype('str')

print(data_VDM_start.shape)
data_VDM_start.head()

In [ ]:
data_VDM_start.groupby(['IdRFID'])['ld_day'].max()

In [ ]:
#some got taken out on 8th day 
data_VDM_start = data_VDM_start[data_VDM_start['ld_day']<8]
data_VDM_start = data_VDM_start[data_VDM_start['ld_day']>0]
data_VDM_start = data_VDM_start[data_VDM_start['VDM_RFID_timediff']<3]
data_VDM_start = data_VDM_start[data_VDM_start['Cage_N']>1]

data_VDM_start['intake_ml_kg'] = data_VDM_start['value'] / (data_VDM_start['weight_pre']/1000)

data_VDM_start.head()

In [ ]:
hour_to_zeit = {0:19,
                1:20,
                2:21,
                3:22,
                4:23,
                5:24,
                6:1,
                7:2,
                8:3,
                9:4,
                10:5,
                11:6,
                12:7,
                13:8,
                14:9,
                15:10,
                16:11,
                17:12,
                18:13,
                19:14,
                20:15,
                21:16,
                22:17,
                23:18,
               }

data_VDM_start['zeitgeber'] = data_VDM_start['VDM_hour'].map(hour_to_zeit)

In [ ]:
len(data_VDM_start[data_VDM_start['Sex']=='female']['IdRFID'].unique())

In [ ]:
len(data_VDM_start[data_VDM_start['Sex']=='male']['IdRFID'].unique())

In [ ]:
data_VDM_start.shape

In [ ]:
data_VDM_total_ld_day_hour = data_VDM_start.groupby(['IdRFID', 'Sex', 'substance', 'ld_cycle', 'zeitgeber', 'ld_day',
                                   ])['intake_ml_kg'].sum().reset_index()

print(data_VDM_total_ld_day_hour.shape)
data_VDM_total_ld_day_hour.head()

In [ ]:
substances = ['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20']
data_final = pd.DataFrame()

for IdRFID in data_VDM_total_ld_day_hour['IdRFID'].unique():
    print(IdRFID)
    
    d_animal = data_VDM_total_ld_day_hour[data_VDM_total_ld_day_hour['IdRFID']==IdRFID]
    
    for substance in substances:
        
        days_hours_df = pd.DataFrame(0, index=np.arange(1,8), columns=data_VDM_start['zeitgeber'].unique()).unstack().reset_index()
        days_hours_df.columns = ['zeitgeber', 'ld_day', 'ddd']
        days_hours_df['ld_cycle'] = ['light' if x<13 else 'dark' for x in days_hours_df['zeitgeber']]
        days_hours_df['substance'] = substance
        days_hours_df['IdRFID'] = IdRFID
        days_hours_df['Sex'] = d_animal['Sex'].values[0]
        
        x = days_hours_df.merge(d_animal, on=['IdRFID', 'Sex', 'ld_day', 'ld_cycle', 'zeitgeber', 'substance'], 
                                how='left').sort_values(['substance', 'ld_day', 'zeitgeber'])
        
        if data_final.empty:
            data_final = x
        else:
            data_final = pd.concat([data_final, x], axis=0)
        
    print(data_final.shape,'\n')

data_final = data_final.fillna(0).sort_values(['IdRFID', 'substance', 'ld_day', 'zeitgeber'])
data_final.head()

## Get RFID data (already processed in other nb)

In [ ]:
path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/polysubstance_paper/data/final/clean/RFID/RFID_poly_final_clean_231103.csv'

data_RFID_start = pd.read_csv(path)
data_RFID_start = pd.DataFrame(data = data_RFID_start)
print(data_RFID_start.shape)
data_RFID_start.head()

In [ ]:
data_RFID_start.groupby(['IdRFID'])['ld_day'].max()

In [ ]:
#some got taken out on 8th day so end at lights on day 8
data_RFID_start = data_RFID_start[data_RFID_start['ld_day']<8]
data_RFID_start = data_RFID_start[data_RFID_start['ld_day']>0]

data_RFID_start = data_RFID_start[data_RFID_start['Cage_N']>1]
print(data_RFID_start.shape)
data_RFID_start.head()

In [ ]:
hour_to_zeit = {0:19,
                1:20,
                2:21,
                3:22,
                4:23,
                5:24,
                6:1,
                7:2,
                8:3,
                9:4,
                10:5,
                11:6,
                12:7,
                13:8,
                14:9,
                15:10,
                16:11,
                17:12,
                18:13,
                19:14,
                20:15,
                21:16,
                22:17,
                23:18,
               }

data_RFID_start['zeitgeber'] = data_RFID_start['RFID_hour'].map(hour_to_zeit)

## Confirm matching RFIDs 

In [ ]:
set(data_RFID_start['IdRFID'].unique()) - set(data_VDM_start['IdRFID'].unique())

In [ ]:
set(data_VDM_start['IdRFID'].unique()) - set(data_RFID_start['IdRFID'].unique())

In [ ]:
sub_map = {'etoh_05': 'EtOH05',
           'etoh_10': 'EtOH10',
           'fent_05': 'Fent05',
           'fent_20': 'Fent20',
           'water_1': 'water',
           'water_2': 'water',}

data_RFID_start['substance'] = data_RFID_start['unitLabel_drink'].map(sub_map)

In [ ]:
data_RFID_total_ld_day_hour = data_RFID_start.groupby(['IdRFID', 'Sex', 'substance', 'ld_cycle', 'zeitgeber', 'ld_day',
                                   ])['eventDuration'].sum().reset_index()

print(data_RFID_total_ld_day_hour.shape)
data_RFID_total_ld_day_hour.head()

In [ ]:
substances = ['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20']
data_RFID_final = pd.DataFrame()

for IdRFID in data_RFID_total_ld_day_hour['IdRFID'].unique():
    print(IdRFID)
    
    d_animal = data_RFID_total_ld_day_hour[data_RFID_total_ld_day_hour['IdRFID']==IdRFID]
    
    for substance in substances:
        
        days_hours_df = pd.DataFrame(0, index=np.arange(1,8), columns=data_RFID_start['zeitgeber'].unique()).unstack().reset_index()
        days_hours_df.columns = ['zeitgeber', 'ld_day', 'ddd']
        days_hours_df['ld_cycle'] = ['light' if x<13 else 'dark' for x in days_hours_df['zeitgeber']]
        days_hours_df['substance'] = substance
        days_hours_df['IdRFID'] = IdRFID
        days_hours_df['Sex'] = d_animal['Sex'].values[0]
        
        x = days_hours_df.merge(d_animal, on=['IdRFID', 'Sex', 'ld_day', 'ld_cycle', 'zeitgeber', 'substance'], 
                                how='left').sort_values(['substance', 'ld_day', 'zeitgeber'])
        
        if data_RFID_final.empty:
            data_RFID_final = x
        else:
            data_RFID_final = pd.concat([data_RFID_final, x], axis=0)
        
    print(data_RFID_final.shape,'\n')

data_RFID_final = data_RFID_final.fillna(0).sort_values(['IdRFID', 'substance', 'ld_day', 'zeitgeber'])
data_RFID_final.head()

In [ ]:
data_RFID = data_RFID_final

## Behav baseline data

In [ ]:
path_behav = '/Users/abbieschindler/Documents/Schindler_Lab/Data/RFID_VDM/polysubstance_paper/meta/RFID_VDM_behavior.xlsx'

data_behav = pd.read_excel(path_behav)
data_behav = pd.DataFrame(data = data_behav)
data_behav = data_behav[data_behav['IdRFID'].isin(data_RFID_total_ld_day_hour['IdRFID'].unique())]
data_behav['Id'] = data_behav['Sex'] + '_' + data_behav['IdRFID'].astype('str')

print(data_behav.shape)
data_behav.head()

### stats on behavior male vs. female

In [ ]:
params = ['EZM_distance', 'EZM_speed', 'EZM_OAE', 'EZM_OAT',
       'EZM_OAD', 'EZM_OAL', 'OFB_distance', 'OFB_speed', 'OFB_CE',
       'OFB_CT', 'OFB_CD', 'OFB_CL']

for param in params:

    g = sns.catplot(x='Sex', y=param, data=data_behav,  kind='bar', 
                    ci=68, height=5, aspect=1, palette='PuBuGn')
    # map data to stripplot
    g.map(sns.stripplot, 'Sex', param,
          palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)
    plt.ylabel(param)
    plt.show()

    male = data_behav[data_behav['Sex']=='male'][param]
    female = data_behav[data_behav['Sex']=='female'][param]
    print(stats.ttest_ind(female, male))

### cor viz

In [ ]:
def corr_sig(df=None):
    p_matrix = np.zeros(shape=(df.shape[1],df.shape[1]))
    for col in df.columns:
        for col2 in df.drop(col,axis=1).columns:
            _ , p = stats.spearmanr(df[col],df[col2])
            p_matrix[df.columns.to_list().index(col),df.columns.to_list().index(col2)] = p
    return p_matrix

def plot_cor_matrix(corr, mask=None):
    f, ax = plt.subplots(figsize=(10,15))
    sns.heatmap(corr, ax=ax,
                mask=mask,
                # cosmetics
                annot=False, vmin=-1, vmax=1, center=0,
                cmap='coolwarm', linewidths=1, linecolor='black', cbar_kws={'orientation': 'horizontal'})


# Plotting with significance filter  
corr = data_behav[params].corr()
p_values = corr_sig(corr)                     # get p-Value
mask = np.invert(np.tril(p_values<0.01))    # mask - only get significant corr
plot_cor_matrix(corr,mask)  

plt.xticks(rotation = 90)
plt.yticks(rotation = 0)
plt.show()

plt.show()


### cluster viz

In [ ]:
sns.color_palette("flare", as_cmap=True)

lut = dict(zip(data_behav['Sex'].unique(), "cb"))
row_colors = data_behav['Sex'].map(lut)
plt.figure(figsize=(15,30))

g = sns.clustermap(data_behav[params].dropna(axis=0), row_colors=row_colors,
                 metric="euclidean", z_score=1, method="ward",
               vmin=-2, vmax=2, center=0, cmap = 'PuBuGn', 
               square=True, cbar_kws={"shrink": .5}, figsize=(10,10))
plt.show()

# Behavior PCA then cluster

## PCA

In [ ]:
#dimensionality reduction
from sklearn.decomposition import PCA

feat_df = data_behav[['EZM_distance', 'EZM_speed', 'EZM_OAE', 'EZM_OAT',
       'EZM_OAD', 'EZM_OAL', 'OFB_distance', 'OFB_speed', 'OFB_CE',
       'OFB_CT', 'OFB_CD', 'OFB_CL',]]

df_st =  StandardScaler().fit_transform(feat_df)  

pca_out = PCA().fit(df_st)

pca_out.explained_variance_ratio_

In [ ]:
exp_var_pca = pca_out.explained_variance_ratio_
#
# Cumulative sum of eigenvalues; This will be used to create step plot
# for visualizing the variance explained by each principal component.
#
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
#
# Create the visualization plot
#
plt.figure(figsize=(10,5))
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual exp var')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative exp var')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')

plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
loadings = pca_out.components_
num_pc = pca_out.n_features_
pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
loadings_df = pd.DataFrame.from_dict(dict(zip(pc_list, loadings)))
loadings_df['substance'] = feat_df.columns.values
loadings_df = loadings_df.set_index('substance')
loadings_df.head()

In [ ]:
plt.figure(figsize=(7,7))
ax = sns.heatmap(loadings_df, annot=False, cmap='PuBuGn')
plt.show()

In [ ]:
# get PC scores
pca_scores = PCA().fit_transform(df_st)

from bioinfokit.visuz import cluster

# get 2D biplot
cluster.biplot(cscore=pca_scores, loadings=loadings, labels=feat_df.columns.values, 
               var1=round(pca_out.explained_variance_ratio_[0]*100, 2),
               var2=round(pca_out.explained_variance_ratio_[1]*100, 2), 
               show=True, dim=(9,13), axlabelfontsize=29,
               colordot=["#A6BDDB", "#014636"],
              colorlist=data_behav['Sex'], legendpos='upper left', dotsize=50)

In [ ]:
pc_list = ["PC"+str(i) for i in list(range(1, num_pc+1))]
id_scores_df = pd.DataFrame.from_dict(pca_scores)
id_scores_df.columns = pc_list
id_scores_df['IdRFID'] = data_behav['IdRFID'].values

print(id_scores_df.shape)
id_scores_df.head()

## Cluster

In [ ]:
data_behav_PCA = data_behav.merge(id_scores_df, on='IdRFID')

print(data_behav_PCA.shape)
data_behav_PCA.head()

In [ ]:
#viz dendrogram to find if three clusters makes sense
plt.figure(figsize=(10, 5))  
plt.title("Dendrogram")  
plt.ylabel("Distance (dissimilarity)")
plt.xlabel("Mice")
dend = shc.dendrogram(shc.linkage(data_behav_PCA[['PC1', 'PC2', 'PC3']], method='ward'), 
                      distance_sort='ascending', 
                      show_leaf_counts=True, leaf_font_size=13)

In [ ]:
def kmeans_stability(data, k):
    
    ####determine cluster stability - bootstrap starting random state - e.g. different cluster initialization 
    
    scores = {}
    
    homogeneity_score_list = []
    completeness_score_list = []
    v_measure_score_list = []
    adjusted_rand_score_list = []
    adjusted_mutual_info_score_list = []
    
    #create initial cluster as baseline comparison
    km_orig = KMeans(n_clusters=k, random_state=39)
    km_orig.fit(data)
    orig_clusters = km_orig.labels_

    #bootstrap random state and compare to baseline cluster
    for i in range(1,99,3):

        #fit
        km_int = KMeans(n_clusters=k, random_state=i)
        km_int.fit(data)
        int_clusters = km_int.labels_
    
        #compute metrics
        homogeneity_score_int = metrics.homogeneity_score(orig_clusters, int_clusters)
        completeness_score_int = metrics.completeness_score(orig_clusters, int_clusters)
        v_measure_score_int = metrics.v_measure_score(orig_clusters, int_clusters)
        adjusted_rand_score_int = metrics.adjusted_rand_score(orig_clusters, int_clusters)
        adjusted_mutual_info_score_int = metrics.adjusted_mutual_info_score(orig_clusters,  int_clusters)
    
        homogeneity_score_list.append(homogeneity_score_int)
        completeness_score_list.append(completeness_score_int)
        v_measure_score_list.append(v_measure_score_int)
        adjusted_rand_score_list.append(adjusted_rand_score_int)
        adjusted_mutual_info_score_list.append(adjusted_mutual_info_score_int)
    
    scores['homogeneity_score'] = homogeneity_score_list
    scores['completeness_score'] = completeness_score_list
    scores['v_measure_score'] = v_measure_score_list
    scores['adjusted_rand_score'] = adjusted_rand_score_list
    scores['adjusted_mutual_info_score'] = adjusted_mutual_info_score_list
    
    return scores

In [ ]:
scores_k2 = pd.DataFrame.from_dict(kmeans_stability(data_behav_PCA[['PC1', 'PC2', 'PC3']], 2)).mean()
scores_k3 = pd.DataFrame.from_dict(kmeans_stability(data_behav_PCA[['PC1', 'PC2', 'PC3']], 3)).mean()
scores_k4 = pd.DataFrame.from_dict(kmeans_stability(data_behav_PCA[['PC1', 'PC2', 'PC3']], 4)).mean()
scores_k5 = pd.DataFrame.from_dict(kmeans_stability(data_behav_PCA[['PC1', 'PC2', 'PC3']], 5)).mean()
scores_k6 = pd.DataFrame.from_dict(kmeans_stability(data_behav_PCA[['PC1', 'PC2', 'PC3']], 6)).mean()

cluster_scores = pd.DataFrame(data=[scores_k2, scores_k3, scores_k4, scores_k5, scores_k6], index=['k=2', 'k=3', 'k=4', 'k=5', 'k=6']).reset_index()

cluster_scores

In [ ]:
sns.lineplot(data=cluster_scores, x='index', y='homogeneity_score')

In [ ]:
#choose k=3 clusters and fit data
km_3 = KMeans(n_clusters=3,random_state=99)
km_3.fit(data_behav_PCA[['PC1', 'PC2', 'PC3']])

data_behav_PCA['kmeans_cluster'] = ["cluster_" + str(label) for label in km_3.labels_ ]
print(data_behav_PCA.shape)
data_behav_PCA.head(1)

In [ ]:
data_behav_PCA.to_csv('data_behav_PCA.csv')

In [ ]:
params = ['EZM_distance', 'EZM_speed', 'EZM_OAE', 'EZM_OAT',
       'EZM_OAD', 'EZM_OAL', 'OFB_distance', 'OFB_speed', 'OFB_CE',
       'OFB_CT', 'OFB_CD', 'OFB_CL']
cluster_order = ['cluster_0', 'cluster_1', 'cluster_2']
for param in params:

    g = sns.catplot(x='kmeans_cluster', y=param, data=data_behav_PCA,  kind='bar', 
                    ci=68, height=5, aspect=1.5, palette='PuBuGn', order=cluster_order)
    # map data to stripplot
    g.map(sns.stripplot, 'kmeans_cluster', param, order=cluster_order,
          palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)
    plt.ylabel(param)
    plt.show()
    
    a=data_behav_PCA[data_behav_PCA['kmeans_cluster']=='cluster_0'][param].values
    b=data_behav_PCA[data_behav_PCA['kmeans_cluster']=='cluster_1'][param].values
    c=data_behav_PCA[data_behav_PCA['kmeans_cluster']=='cluster_2'][param].values
    print(f_oneway(a, b, c))

In [ ]:
plt.figure(figsize=(7, 5))  

sns.countplot(x='kmeans_cluster', data=data_behav_PCA, hue='Sex', palette="PuBuGn", order=cluster_order)

In [ ]:
data_behav_PCA_count = data_behav_PCA.groupby(['Sex'])['kmeans_cluster'].value_counts().reset_index(name='count')

data_behav_PCA_count2 = data_behav_PCA['Sex'].value_counts().reset_index(name='count')

data_behav_PCA_count = data_behav_PCA_count.merge(data_behav_PCA_count2, left_on='Sex', right_on='index')

data_behav_PCA_count['kmeans_perc'] = data_behav_PCA_count['count_x'] / data_behav_PCA_count['count_y'] * 100

data_behav_PCA_count

In [ ]:
data_behav_PCA_count.sort_values(['Sex', 'kmeans_cluster'])

In [ ]:
table = np.array([[4, 5, 15], [7, 1, 24]])
chi2_contingency(table)


In [ ]:
g = sns.catplot(x='kmeans_cluster', y='kmeans_perc', data=data_behav_PCA_count,  kind='bar', hue='Sex',
                    ci=68, height=5, aspect=2, palette='PuBuGn', order=cluster_order)
plt.legend(loc='upper left')
plt.ylabel('Percent in Cluster')
plt.show()

# Project clusters back onto RFID data

In [ ]:
cluster_order = ['cluster_0', 'cluster_1', 'cluster_2']
drink_order=['water', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20']


In [ ]:
data_RFID = data_behav_PCA[['IdRFID', 'kmeans_cluster']].merge(data_RFID, on='IdRFID')
data_RFID_start = data_behav_PCA[['IdRFID', 'kmeans_cluster']].merge(data_RFID_start, on='IdRFID')

data_RFID.head()

## total with combined substances

### event duration proportion

In [ ]:
sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='kmeans_cluster', palette='PuBuGn',
            hue_order=cluster_order)
#plt.ylim(0,2000)
plt.xlim(0,30)
plt.xlabel('Time in chamber (sec)')

plt.show()

### total event duration sum

In [ ]:
data_RFID_total = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total['Time in chamber (min)'] = data_RFID_total['eventDuration']/60

data_RFID_total.head()

In [ ]:
g = sns.catplot(x='kmeans_cluster', y='Time in chamber (min)', data=data_RFID_total,  kind='bar', height=5, aspect=1.5,
            ci=68, palette='PuBuGn', order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'kmeans_cluster', 'Time in chamber (min)', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, order=cluster_order)

plt.show()

param = 'Time in chamber (min)'
a=data_RFID_total[data_RFID_total['kmeans_cluster']=='cluster_0'][param].values
b=data_RFID_total[data_RFID_total['kmeans_cluster']=='cluster_1'][param].values
c=data_RFID_total[data_RFID_total['kmeans_cluster']=='cluster_2'][param].values
print(f_oneway(a, b, c))

## light dark with combined substances

### event duration proportion

In [ ]:
g=sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='ld_cycle', 
              col='kmeans_cluster', palette='PuBuGn', col_order=cluster_order)
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_total_ld = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 'ld_cycle', 
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_ld['Time in chamber (min)'] = data_RFID_total_ld['eventDuration']/60

data_RFID_total_ld.head()

In [ ]:
g= sns.catplot(x='ld_cycle', y='Time in chamber (min)', data=data_RFID_total_ld,  kind='bar', height=5, aspect=1,
            ci=68, hue='kmeans_cluster', palette='PuBuGn', hue_order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'ld_cycle', 'Time in chamber (min)', 'kmeans_cluster', hue_order=cluster_order,
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)

plt.xlabel('Light/Dark Cycle')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_cycle', between='kmeans_cluster', subject='IdRFID', data=data_RFID_total_ld)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_cycle', between='kmeans_cluster',
                              subject='IdRFID', data=data_RFID_total_ld, padjust='holm')
pg.print_table(posthocs)

## daily total with combined substance 

### event duration proportion

In [ ]:
g=sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='ld_day', col='kmeans_cluster', 
              palette='PuBuGn', col_order=cluster_order)
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_total_daily = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 'ld_day',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_daily['Time in chamber (min)'] = data_RFID_total_daily['eventDuration']/60

data_RFID_total_daily.head()

In [ ]:
g= sns.catplot(x='ld_day', y='Time in chamber (min)', data=data_RFID_total_daily,  kind='bar', height=5, aspect=2,
            ci=68, hue='kmeans_cluster', palette='PuBuGn', hue_order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'ld_day', 'Time in chamber (min)', 'kmeans_cluster',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, hue_order=cluster_order)

plt.xlabel('Day')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='kmeans_cluster', subject='IdRFID', data=data_RFID_total_daily)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='kmeans_cluster',
                              subject='IdRFID', data=data_RFID_total_daily, padjust='holm')
pg.print_table(posthocs)

## hourly total with combined substances

### event duration proportion

In [ ]:
g=sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='zeitgeber', col='kmeans_cluster', 
              palette='PuBuGn', col_order=cluster_order)
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_total_hourly = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 'zeitgeber', 
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_hourly['Time in chamber (min)'] = data_RFID_total_hourly['eventDuration']/60

data_RFID_total_hourly.head()

In [ ]:
g= sns.catplot(x='zeitgeber', y='Time in chamber (min)', data=data_RFID_total_hourly,  kind='bar', height=5, aspect=2,
            ci=68, hue='kmeans_cluster', palette='PuBuGn', hue_order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'zeitgeber', 'Time in chamber (min)', 'kmeans_cluster', hue_order=cluster_order,
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1,
      )

plt.xlabel('Zeitgeber time (hour)')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='zeitgeber', between='kmeans_cluster', subject='IdRFID', data=data_RFID_total_hourly)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='zeitgeber', between='kmeans_cluster',
                              subject='IdRFID', data=data_RFID_total_hourly, padjust='holm')
pg.print_table(posthocs)

## daily and hourly combined substances heatmap

In [ ]:
data_RFID_total_hour_day = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 'zeitgeber', 'ld_day',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_hour_day['Time in chamber (min)'] = data_RFID_total_hour_day['eventDuration']/60

data_RFID_total_hour_day.head()

### event duration

In [ ]:
d = data_RFID_total_hour_day[data_RFID_total_hour_day['kmeans_cluster']=='cluster_2']
groupby = d.groupby(['ld_day', 'zeitgeber'])['Time in chamber (min)'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "Time in chamber (min)")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=12,
                     cbar_kws={'label': 'Time in chamber (min)'})

#plt.xlabel('Bottle')
plt.show()

## total with separate substances 

### event duration proportion

In [ ]:

g=sns.displot(data=data_RFID_start, x="eventDuration", kind='ecdf', hue='substance', col='kmeans_cluster', 
              palette='PuBuGn', hue_order=drink_order, col_order=cluster_order)
#plt.ylim(0,2000)
plt.xlim(0,30)

g.set_axis_labels('Time in chamber (sec)')
plt.show()

### total event duration sum

In [ ]:
data_RFID_total_sub = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 'substance',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_sub['Time in chamber (min)'] = data_RFID_total_sub['eventDuration']/60

data_RFID_total_sub.head()

In [ ]:
g= sns.catplot(x='kmeans_cluster', y='Time in chamber (min)', data=data_RFID_total_sub,  kind='bar', height=5, aspect=2,
            ci=68, hue='substance', hue_order=drink_order, order=cluster_order, palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'kmeans_cluster', 'Time in chamber (min)', 'substance',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1,
      hue_order=drink_order, order=cluster_order,)

#plt.xlabel('Cluster')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='substance', between='kmeans_cluster', subject='IdRFID', data=data_RFID_total_sub)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='substance', between='kmeans_cluster',
                              subject='IdRFID', data=data_RFID_total_sub, padjust='fdr_bh')
pg.print_table(posthocs)

## light dark with separate substances 

### event duration proportion

### total event duration sum

In [ ]:
data_RFID_sub_ld = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 'ld_cycle', 'substance',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_sub_ld['Time in chamber (min)'] = data_RFID_sub_ld['eventDuration']/60

data_RFID_sub_ld.head()

In [ ]:
x = data_RFID_sub_ld[data_RFID_sub_ld['kmeans_cluster']=='cluster_1'].set_index(['kmeans_cluster', 'ld_cycle', 'IdRFID',  'substance', ]).unstack(-1).reset_index()
x.head()

In [ ]:
x.to_csv('data_RFID_sub_ld_cluster1.csv')

In [ ]:
d = data_RFID_sub_ld[data_RFID_sub_ld['kmeans_cluster']=='cluster_2']

g= sns.catplot(x='ld_cycle', y='Time in chamber (min)', data=d,  kind='bar', height=5, aspect=1,
            ci=68, hue='substance', palette='PuBuGn', hue_order=drink_order)

# map data to stripplot
g.map(sns.stripplot, 'ld_cycle', 'Time in chamber (min)', 'substance', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', 
      linewidth=1, hue_order=drink_order)

plt.ylim(0,500)
plt.xlabel('Light/Dark Cycle')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.rm_anova(dv=dv, within=['ld_cycle', 'substance'], subject='IdRFID', data=d)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within=['ld_cycle', 'substance'],
                              subject='IdRFID', data=d, padjust='fdr_bh')
pg.print_table(posthocs)

## daily with separate substances

In [ ]:
data_RFID_sub_daily = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 'ld_day', 'substance'
                                   ])['eventDuration'].sum().reset_index()

data_RFID_sub_daily['Time in chamber (min)'] = data_RFID_sub_daily['eventDuration']/60

data_RFID_sub_daily.head()

In [ ]:
data_RFID_sub_daily['Time in chamber (min)'].max()

In [ ]:
sub = 'Fent20'
d = data_RFID_sub_daily[data_RFID_sub_daily['substance']==sub]
order=['cluster_0', 'cluster_1', 'cluster_2']
g= sns.catplot(x='kmeans_cluster', y='Time in chamber (min)', data=d,  kind='bar', height=5, aspect=1.2,
            ci=68, hue='ld_day', palette='PuBuGn', order=order)

# map data to stripplot
g.map(sns.stripplot, 'kmeans_cluster', 'Time in chamber (min)', 'ld_day', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, order=order)

plt.ylim(0,200)
plt.title(sub)
plt.xlabel('Day')
plt.show()

In [ ]:
dv = 'Time in chamber (min)'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='kmeans_cluster', subject='IdRFID', data=d)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='kmeans_cluster',
                              subject='IdRFID', data=d, padjust='fdr_bh')
pg.print_table(posthocs)

## daily and hourly total with separate substance heatmap

### event duration

In [ ]:
data_RFID_total_sub_day_hour = data_RFID.groupby(['IdRFID', 'kmeans_cluster', 'substance', 'ld_day', 'zeitgeber',
                                   ])['eventDuration'].sum().reset_index()

data_RFID_total_sub_day_hour['Time in chamber (min)'] = data_RFID_total_sub_day_hour['eventDuration']/60

data_RFID_total_sub_day_hour.head()

In [ ]:
data_RFID_total_sub_day_hour['substance'].unique()

In [ ]:
d = data_RFID_total_sub_day_hour[data_RFID_total_sub_day_hour['kmeans_cluster']=='cluster_2']
d = d[d['substance']=='Fent20']

groupby = d.groupby(['ld_day', 'zeitgeber'])['Time in chamber (min)'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "Time in chamber (min)")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=5,
                     cbar_kws={'label': 'Time in chamber (min)'})

#plt.xlabel('Bottle')
plt.show()

## ID heatmap

In [ ]:
data_RFID_total_sub['Id'] = data_RFID_total_sub['kmeans_cluster'] + '_' + data_RFID_total_sub['IdRFID'].astype('str')
data_RFID_total_sub.head()

In [ ]:
sns.set(font_scale=2)
groupby = data_RFID_total_sub.pivot('Id', 'substance', "Time in chamber (min)")
plt.figure(figsize=(5,15))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=500,
                     cbar_kws={'label': 'Time in chamber (min)'})

#plt.xlabel('Bottle')
plt.show()

# Project clusters back onto drinking data

In [ ]:
sns.set_context("poster")
sns.set_style("ticks")
sns.set(font_scale=2)

In [ ]:
data_final_clean_3s = data_final


In [ ]:
data_final_clean_3s = data_behav_PCA[['IdRFID', 'kmeans_cluster']].merge(data_final_clean_3s, on='IdRFID')
data_VDM_start = data_behav_PCA[['IdRFID', 'kmeans_cluster']].merge(data_VDM_start, on='IdRFID')

data_final_clean_3s.head()

## total with combined substances

### total intake sum min

In [ ]:
data_VDM_total_min = data_VDM_start.groupby(['IdRFID', 'kmeans_cluster', 'VDM_min_count_running',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_min.head()

In [ ]:
sns.displot(data=data_VDM_total_min, x="intake_ml_kg", kind='ecdf', hue='kmeans_cluster',
           palette='PuBuGn', hue_order=cluster_order)

plt.xlabel('Intake (ml/kg) per minute')

plt.xlim(0,10)

### total intake sum

In [ ]:
data_VDM_total = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total.head()

In [ ]:
g = sns.catplot(x='kmeans_cluster', y='intake_ml_kg', data=data_VDM_total,  kind='bar', height=5, aspect=1.5,
            ci=68, palette='PuBuGn', order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'kmeans_cluster', 'intake_ml_kg', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, order=cluster_order)
plt.ylabel('Total intake (ml/kg)')
plt.show()

param = 'intake_ml_kg'
a=data_VDM_total[data_VDM_total['kmeans_cluster']=='cluster_0'][param].values
b=data_VDM_total[data_VDM_total['kmeans_cluster']=='cluster_1'][param].values
c=data_VDM_total[data_VDM_total['kmeans_cluster']=='cluster_2'][param].values
print(f_oneway(a, b, c))

## light dark with combined substances

### total intake sum min

In [ ]:
data_VDM_total_ld_min = data_VDM_start.groupby(['IdRFID', 'kmeans_cluster', 'VDM_min_count_running', 'ld_cycle',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_ld_min.head()

In [ ]:
g=sns.displot(data=data_VDM_total_ld_min, x="intake_ml_kg", kind='ecdf', hue='ld_cycle', col='kmeans_cluster', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,10)

g.set_axis_labels('Intake (ml/kg) per minute')
plt.show()

### total intake sum

In [ ]:
data_VDM_total_ld = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 'ld_cycle',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_ld.head()

In [ ]:
g= sns.catplot(x='ld_cycle', y='intake_ml_kg', data=data_VDM_total_ld,  kind='bar', height=5, aspect=1,
            ci=68, hue='kmeans_cluster', palette='PuBuGn', hue_order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'ld_cycle', 'intake_ml_kg', 'kmeans_cluster',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, hue_order=cluster_order)


plt.xlabel('Light/Dark Cycle')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_cycle', between='kmeans_cluster', subject='IdRFID', data=data_VDM_total_ld)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_cycle', between='kmeans_cluster',
                              subject='IdRFID', data=data_VDM_total_ld, padjust='holm')
pg.print_table(posthocs)

## daily total combine substances

### total intake sum min

In [ ]:
data_VDM_total_daily_min = data_VDM_start.groupby(['IdRFID', 'kmeans_cluster', 'VDM_min_count_running', 'ld_day',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_daily_min.head()

In [ ]:
g=sns.displot(data=data_VDM_total_daily_min, x="intake_ml_kg", kind='ecdf', hue='ld_day', col='kmeans_cluster', palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,10)

g.set_axis_labels('Intake (ml/kg) per minute')
plt.show()

### total intake sum

In [ ]:
data_VDM_total_daily = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 'ld_day',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_daily.head()

In [ ]:
g= sns.catplot(x='ld_day', y='intake_ml_kg', data=data_VDM_total_daily,  kind='bar', height=5, aspect=2,
            ci=68, hue='kmeans_cluster', palette='PuBuGn', hue_order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'ld_day', 'intake_ml_kg', 'kmeans_cluster',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, hue_order=cluster_order)

plt.xlabel('Day')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='kmeans_cluster', subject='IdRFID', data=data_VDM_total_daily)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='kmeans_cluster',
                              subject='IdRFID', data=data_VDM_total_daily, padjust='holm')
pg.print_table(posthocs)

## hourly total with combined substances

### total intake sum

In [ ]:
data_VDM_total_hourly = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 'zeitgeber',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_hourly.head()

In [ ]:
g= sns.catplot(x='zeitgeber', y='intake_ml_kg', data=data_VDM_total_hourly,  kind='bar', height=5, aspect=2,
            ci=68, hue='kmeans_cluster', palette='PuBuGn', hue_order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'zeitgeber', 'intake_ml_kg', 'kmeans_cluster',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, hue_order=cluster_order)

plt.xlabel('Zeitgeber time (hour)')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='zeitgeber', between='kmeans_cluster', subject='IdRFID', data=data_VDM_total_hourly)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='zeitgeber', between='kmeans_cluster',
                              subject='IdRFID', data=data_VDM_total_hourly, padjust='holm')
pg.print_table(posthocs)

## daily and hourly combines substances heatmap

### total drinking

In [ ]:
data_VDM_total_hour_day = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 'zeitgeber', 'ld_day',
                                   ])['intake_ml_kg'].sum().reset_index()

data_VDM_total_hour_day.head()

In [ ]:
d = data_VDM_total_hour_day[data_VDM_total_hour_day['kmeans_cluster']=='cluster_2']
groupby = d.groupby(['ld_day', 'zeitgeber'])['intake_ml_kg'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "intake_ml_kg")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=15,
                     cbar_kws={'label': 'Total intake (ml/kg)'})

#plt.xlabel('Bottle')
plt.show()

## total with separate substances

### total intake sum min

In [ ]:
data_VDM_total_sub_min = data_VDM_start.groupby(['IdRFID', 'kmeans_cluster', 'VDM_min_count_running', 'substance',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub_min.head()

In [ ]:
g=sns.displot(data=data_VDM_total_sub_min, x="intake_ml_kg", kind='ecdf', hue='substance', col='kmeans_cluster', 
              hue_order=drink_order, col_order=cluster_order, palette='PuBuGn')
#plt.ylim(0,2000)
plt.xlim(0,10)

g.set_axis_labels('Intake (ml/kg) per minute')
plt.show()

### total intake sum

In [ ]:
data_VDM_total_sub = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 'substance',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub.tail()

In [ ]:
g= sns.catplot(x='kmeans_cluster', y='intake_ml_kg', data=data_VDM_total_sub,  kind='bar', height=5, aspect=2,
            ci=68, hue='substance', hue_order=drink_order, order=cluster_order, palette='PuBuGn')

# map data to stripplot
g.map(sns.stripplot, 'kmeans_cluster', 'intake_ml_kg', 'substance',
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1,
      hue_order=drink_order, order=cluster_order,)

plt.xlabel('Chamber substance')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='substance', between='kmeans_cluster', subject='IdRFID', data=data_VDM_total_sub)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='substance', between='kmeans_cluster',
                              subject='IdRFID', data=data_VDM_total_sub, padjust='fdr_bh')
pg.print_table(posthocs)

## light dark with separate substances 

### total intake

In [ ]:
data_VDM_total_sub_ld = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 'ld_cycle', 'substance',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub_ld.head()

In [ ]:
d = data_VDM_total_sub_ld[data_VDM_total_sub_ld['kmeans_cluster']=='cluster_2']

g= sns.catplot(x='ld_cycle', y='intake_ml_kg', data=d,  kind='bar', height=5, aspect=1,
            ci=68, hue='substance', palette='PuBuGn', hue_order=drink_order,)

# map data to stripplot
g.map(sns.stripplot, 'ld_cycle', 'intake_ml_kg', 'substance', hue_order=drink_order,
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', 
      linewidth=1)

plt.ylim(0,1250)
plt.xlabel('Light/Dark Cycle')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.rm_anova(dv=dv, within=['ld_cycle', 'substance'], subject='IdRFID', data=d)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within=['ld_cycle', 'substance'],
                              subject='IdRFID', data=d, padjust='fdr_bh')
pg.print_table(posthocs)

## daily with separate substances

In [ ]:
data_VDM_total_sub_day = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 'ld_day', 'substance',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub_day.head()

In [ ]:
data_VDM_total_sub_day['intake_ml_kg'].max()

In [ ]:
sub = 'Fent20'
d = data_VDM_total_sub_day[data_VDM_total_sub_day['substance']==sub]
order=['cluster_0', 'cluster_1', 'cluster_2']
g= sns.catplot(x='kmeans_cluster', y='intake_ml_kg', data=d,  kind='bar', height=5, aspect=1.2,
            ci=68, hue='ld_day', palette='PuBuGn', order=order)

# map data to stripplot
g.map(sns.stripplot, 'kmeans_cluster', 'intake_ml_kg', 'ld_day', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, order=order)

plt.ylim(0,400)
plt.title(sub)
plt.xlabel('Day')
plt.ylabel('Total intake (ml/kg)')
plt.show()

In [ ]:
dv = 'intake_ml_kg'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='kmeans_cluster', subject='IdRFID', data=d)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='kmeans_cluster',
                              subject='IdRFID', data=d, padjust='fdr_bh')
pg.print_table(posthocs)

## daily and hourly total with separate substances heatmap

### total intake

In [ ]:
data_VDM_total_sub_day_hour = data_final_clean_3s.groupby(['IdRFID', 'kmeans_cluster', 'substance', 'ld_day', 'zeitgeber',
                                   ])[['intake_ml_kg']].sum().reset_index()

data_VDM_total_sub_day_hour.head()

In [ ]:
d = data_VDM_total_sub_day_hour[data_VDM_total_sub_day_hour['kmeans_cluster']=='cluster_2']
d = d[d['substance']=='Fent20']
groupby = d.groupby(['ld_day', 'zeitgeber'])['intake_ml_kg'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "intake_ml_kg")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=10,
                     cbar_kws={'label': 'Total intake (ml/kg)'})

#plt.xlabel('Bottle')
plt.show()

## preference

#### total by substance

In [ ]:
data_summary_vol = data_VDM_total_sub.set_index(['IdRFID', 'kmeans_cluster', 
                                  'substance'])[['intake_ml_kg']].unstack(-1).reset_index()

data_summary_vol.columns = ['IdRFID', 'kmeans_cluster', 
                        'EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water',]

data_summary_vol.replace(np.nan, 0, inplace=True)

data_summary_vol['total_alcohol_intake'] = data_summary_vol['EtOH05'] + data_summary_vol['EtOH10'] 
data_summary_vol['total_fent_intake'] = data_summary_vol['Fent05'] + data_summary_vol['Fent20'] 

data_summary_vol['overall_alcohol_pref'] = data_summary_vol['total_alcohol_intake'] / (data_summary_vol['total_alcohol_intake'] + data_summary_vol['water'])
data_summary_vol['overall_fent_pref'] = data_summary_vol['total_fent_intake'] / (data_summary_vol['total_fent_intake'] + data_summary_vol['water'])

data_summary_vol['DP_alcohol'] =  ((data_summary_vol['EtOH05']*.05) + (data_summary_vol['EtOH10']*.1))  \
/ (data_summary_vol['EtOH05'] + data_summary_vol['EtOH10'])

data_summary_vol['DP_fent'] =  ((data_summary_vol['Fent05']*.05) + (data_summary_vol['Fent20']*.2))  \
/ (data_summary_vol['Fent05'] + data_summary_vol['Fent20'])
data_summary_vol.replace(np.nan, 0, inplace=True)
data_summary_vol.head()

In [ ]:
g = sns.catplot(x='kmeans_cluster', y='DP_fent', data=data_summary_vol,  kind='bar', height=5, aspect=1.2,
            ci=68, palette='PuBuGn', order=cluster_order)

# map data to stripplot
g.map(sns.stripplot, 'kmeans_cluster', 'DP_fent', 
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1, order=cluster_order)
plt.ylabel('Fentanyl Dose Preference')
plt.show()

param = 'DP_fent'
a=data_summary_vol[data_summary_vol['kmeans_cluster']=='cluster_0'][param].values
b=data_summary_vol[data_summary_vol['kmeans_cluster']=='cluster_1'][param].values
c=data_summary_vol[data_summary_vol['kmeans_cluster']=='cluster_2'][param].values
print(f_oneway(a, b, c))

#### daily by substance

In [ ]:
data_summary_vol_day = data_VDM_total_sub_day.set_index(['IdRFID', 'kmeans_cluster', 'ld_day', 
                                  'substance'])[['intake_ml_kg']].unstack(-1).reset_index()

data_summary_vol_day.columns = ['IdRFID', 'kmeans_cluster', 'ld_day', 
                        'EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water',]

data_summary_vol_day.replace(np.nan, 0, inplace=True)

data_summary_vol_day['total_alcohol_intake'] = data_summary_vol_day['EtOH05'] + data_summary_vol_day['EtOH10'] 
data_summary_vol_day['total_fent_intake'] = data_summary_vol_day['Fent05'] + data_summary_vol_day['Fent20'] 

data_summary_vol_day['overall_alcohol_pref'] = data_summary_vol_day['total_alcohol_intake'] / (data_summary_vol_day['total_alcohol_intake'] + data_summary_vol_day['water'])
data_summary_vol_day['overall_fent_pref'] = data_summary_vol_day['total_fent_intake'] / (data_summary_vol_day['total_fent_intake'] + data_summary_vol_day['water'])

data_summary_vol_day['DP_alcohol'] =  ((data_summary_vol_day['EtOH05']*.05) + (data_summary_vol_day['EtOH10']*.1))  \
/ (data_summary_vol_day['EtOH05'] + data_summary_vol_day['EtOH10'])

data_summary_vol_day['DP_fent'] =  ((data_summary_vol_day['Fent05']*.05) + (data_summary_vol_day['Fent20']*.2))  \
/ (data_summary_vol_day['Fent05'] + data_summary_vol_day['Fent20'])

data_summary_vol_day.replace(np.nan, 0, inplace=True)
data_summary_vol_day.head()

In [ ]:
g= sns.catplot(x='ld_day', y='overall_fent_pref', data=data_summary_vol_day,  kind='bar', height=5, aspect=1,
            ci=68, hue='kmeans_cluster', palette='PuBuGn', hue_order=cluster_order)

# map data to DP_fent
g.map(sns.stripplot, 'ld_day', 'overall_fent_pref', 'kmeans_cluster', hue_order=cluster_order,
      palette=sns.color_palette('PuBuGn')[2:4], dodge=True, alpha=0.6, ec='k', linewidth=1)

#plt.ylim(0,1250)
plt.xlabel('Day')
plt.ylabel('Fentanyl/Water Preference')
plt.show()

In [ ]:
dv = 'DP_fent'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='kmeans_cluster', subject='IdRFID', data=data_summary_vol_day)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='kmeans_cluster',
                              subject='IdRFID', data=data_summary_vol_day, padjust='fdr_bh')
pg.print_table(posthocs)

#### daily and hourly total by substance

In [ ]:
data_summary_vol_day = data_VDM_total_sub_day_hour.set_index(['IdRFID', 'kmeans_cluster', 'ld_day', 'zeitgeber',
                                  'substance'])[['intake_ml_kg']].unstack(-1).reset_index()

data_summary_vol_day.columns = ['IdRFID', 'kmeans_cluster', 'ld_day', 'zeitgeber',
                        'EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water',]

data_summary_vol_day.replace(np.nan, 0, inplace=True)

data_summary_vol_day['total_alcohol_intake'] = data_summary_vol_day['EtOH05'] + data_summary_vol_day['EtOH10'] 
data_summary_vol_day['total_fent_intake'] = data_summary_vol_day['Fent05'] + data_summary_vol_day['Fent20'] 

data_summary_vol_day['overall_alcohol_pref'] = data_summary_vol_day['total_alcohol_intake'] / (data_summary_vol_day['total_alcohol_intake'] + data_summary_vol_day['water'])
data_summary_vol_day['overall_fent_pref'] = data_summary_vol_day['total_fent_intake'] / (data_summary_vol_day['total_fent_intake'] + data_summary_vol_day['water'])

data_summary_vol_day['DP_alcohol'] =  ((data_summary_vol_day['EtOH05']*.05) + (data_summary_vol_day['EtOH10']*.1))  \
/ (data_summary_vol_day['EtOH05'] + data_summary_vol_day['EtOH10'])

data_summary_vol_day['DP_fent'] =  ((data_summary_vol_day['Fent05']*.05) + (data_summary_vol_day['Fent20']*.2))  \
/ (data_summary_vol_day['Fent05'] + data_summary_vol_day['Fent20'])

data_summary_vol_day.replace(np.nan, 0, inplace=True)
data_summary_vol_day.head()

In [ ]:
d = data_summary_vol_day[data_summary_vol_day['kmeans_cluster']=='cluster_2']

groupby = d.groupby(['ld_day', 'zeitgeber'])['DP_fent'].mean().reset_index()


sns.set(font_scale=2)
groupby = groupby.pivot('ld_day', 'zeitgeber', "DP_fent")
plt.figure(figsize=(5,5))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, vmax=.2,
                     cbar_kws={'label': 'Fentanyl Preference'})

#plt.xlabel('Bottle')
plt.show()

In [ ]:
dv = 'overall_alcohol_pref'

#RM using Pingouin
aov = pg.mixed_anova(dv=dv, within='ld_day', between='kmeans_cluster', subject='IdRFID', data=data_summary_vol_day)
pg.print_table(aov)

#posthocs
posthocs = pg.pairwise_ttests(dv=dv, within='ld_day', between='kmeans_cluster',
                              subject='IdRFID', data=data_summary_vol_day, padjust='holm')
pg.print_table(posthocs)

## ID heatmap

In [ ]:
data_VDM_total_sub['Id'] = data_VDM_total_sub['kmeans_cluster'] + '_' + data_VDM_total_sub['IdRFID'].astype('str')
data_VDM_total_sub.head()

In [ ]:
sns.set(font_scale=2)
groupby = data_VDM_total_sub.pivot('Id', 'substance', "intake_ml_kg")
plt.figure(figsize=(5,15))
ax = sns.heatmap(groupby, cmap="PuBuGn", linewidths=.25, 
                     cbar_kws={'label': 'Total intake (ml/kg)'})

#plt.xlabel('Bottle')
plt.show()